Connecting Notebook to Datebase

In [1]:
# Use this to download any necessary modules
#import sys
#!{sys.executable} -m pip install psycopg2
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install sqlalchemy
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install scikit_posthocs
#!{sys.executable} -m pip install termcolor
#!{sys.executable} -m pip install warnings
#!{sys.executable} -m pip install plotnine
#!{sys.executable} -m pip install json

In [2]:
import psycopg2 #for database connection
import pandas as pd 
import sqlalchemy
import numpy as np
import matplotlib.pyplot as plt # for plotting
import json

# Kruskal-Wallis analysis of variance
import scipy.stats as ss # For Kruskal-Wallis test
import scikit_posthocs as sp #For post hoc tests. 
from termcolor import colored # for coloring the print text
import warnings # to ignore plot warnings

# For ggplot
from plotnine import *

# For bolding the printed text
from termcolor import colored


In [4]:
# Create Connection
try:
    connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database, port=port )
    
    
except:
    print("I am unable to connect to the database")

In [5]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 32)

Beginning of Version 1.00

In [ ]:
# Step 1
# Getting gold standard images from phase 1
gold_std = pd.read_sql_query("select subject_id_ph1 as subject_id, filename, malicious, gold_std from cybertrust_zooniverse_datamatch where gold_std is true", connection)
gold_std

gold_std = gold_std.dropna(subset= ['subject_id']).reset_index()
gold_std = gold_std.drop('index', axis=1)
gold_std


In [ ]:
# Step 2
# Getting subject_id and raw json strings that hold the user's bounding boxes
# The code technically gets all the entries where a user select that an image looked fishy.
# It does this by getting the subject id and the bounding boxes drawn by a user for that subject id.

# If you see an entry in the table below that has "[]" in the user_answer column,
# that would be an instance where the user answered that the image was phishy, but did not draw any bounding boxes

sql = """select c.subject_ids as subject_id, a.annotations->>'value' as user_answers, b.annotations->> 'value' as phishing_class 
       from zooniverse_phish_classifications as c, 
    jsonb_array_elements(c.annotations) as a(annotations),
    jsonb_array_elements(c.annotations) as b(annotations)
    where a.annotations->>'task' = 'T0' and 
        b.annotations->>'task' = 'T3'
"""

zooniverseclassification = pd.read_sql_query(sql, connection)
zooniverseclassification

#zooniverseclassification = zooniverseclassification[zooniverseclassification['user_answers'] == '[]']
#zooniverseclassification[zooniverseclassification['subject_id'] == 43857067]
zooniverseclassification


In [ ]:

Zoon_malicious= zooniverseclassification.groupby('subject_id').apply(lambda x: pd.Series(
    dict( Total_People_who_put_malicious = (x.phishing_class == "Something's Phishy").sum()
    ))).reset_index('subject_id')

Zoon_malicious

change_type ={"subject_id": "int64"}
Zoon_malicious = Zoon_malicious.astype(change_type)
data_types = Zoon_malicious.dtypes
data_types

Zoon_malicious = pd.merge(gold_std, Zoon_malicious, on="subject_id")
Zoon_malicious



#Zoon_malicious.to_csv("Zoon_malicious.csv", index=False)

In [ ]:
No_cue_type= zooniverseclassification[zooniverseclassification['user_answers'] == '[]']
Count_No_Cue_Type = No_cue_type.groupby('subject_id').apply(lambda x: pd.Series(
    dict(User_who_did_not_put_cue = (x.user_answers == '[]').sum()
    ))).reset_index('subject_id')

Count_No_Cue_Type= pd.DataFrame(Count_No_Cue_Type)
Count_No_Cue_Type



In [ ]:
change_type ={"subject_id": "int64"}
Count_No_Cue_Type = Count_No_Cue_Type.astype(change_type)
data_types = Count_No_Cue_Type.dtypes
data_types

No_Cue_Gold_Standard = pd.merge(gold_std, Count_No_Cue_Type, on="subject_id")
No_Cue_Gold_Standard

#No_Cue_Gold_Standard.to_csv("No_Cue_Gold_Standard.csv", index=False)

In [12]:
# Step 3 
# Sorting Dataframe separting the different cues given in json format
# This dataframe stores all the information related to the bounded boxes.
# User_labeled_image is one user's session of labeling one certain image to search for unique cues given by any user
usercues = pd.DataFrame(columns=["user_labeled_image", "subject_id", "cue"])

In [13]:
# This could takes all user bounding boxes and sorts them by 
# user_labeled_image(unique user interaction with unique image), 
# subject_id(image), and cues.
# This doesn't not filter out duplicate cue from a user on a certain image

for user_labeled_image in range(0, len(zooniverseclassification["user_answers"])):
    current_string = zooniverseclassification["user_answers"][user_labeled_image]
    json_object = json.loads(current_string)
    
    for item in json_object:
        data_input = {"user_labeled_image": user_labeled_image, 
                            "subject_id": zooniverseclassification["subject_id"][user_labeled_image], 
                            "cue": item["tool_label"]}   
        
        usercues.loc[len(usercues.index)] = data_input

In [ ]:
# Dataframe made from code above
# One error "user_labeled_image" for 0 is repeated
usercues

unique_terms = usercues['cue'].unique()
unique_terms


In [16]:
pd.set_option('display.max_rows', None)

In [ ]:
# Step 4
# Filtering out duplicate cue given by a single user
usercues = usercues.drop_duplicates()

check = usercues[usercues['subject_id'].isin(['49251699', '49251697', '49251696', '49251694', '49251678', '49251693',
'49251691',
'49251690',
'49251687',
'49251675',
'49251686',
'49251683',
'49251682',
'49251681',
'49251704',
'49251703',
'49251702',
'49251701',
'49251700',
'49251698',
'49251695',
'49251677',
'49251692',
'49251676',
'49251689',
'49251688',
'49251685',
'49251684',
'49251680',
'49251679'
])]

check[check['cue'].isin(['Appeal to Action-Urgency'])]

In [ ]:
# Step 5 
# Getting the total number of cues of each type sorted by the image
# Sorting the cues given by unique users
sortedcues = usercues.groupby('subject_id').apply(lambda x: pd.Series(
    dict(Invalid_Domain_or_Sender = (x.cue == "Invalid Domain or Sender").sum(),
         Potent_Mal_Links = (x.cue == "Potentially Malicious Link").sum(),
         Spelling_or_Grammar = (x.cue == "Poor Spelling or Grammar").sum(),
         Appeal_to_Greed = (x.cue == "Appeal to Action-Greed").sum(),
         Appeal_to_Urgency = (x.cue == "Appeal to Action-Urgency").sum(),
         Appeal_to_Authority = (x.cue == "Appeal to Action-Authority").sum(),
         Other_Phishy_Findings = (x.cue == "Other Phishy Findings").sum()
    ))).reset_index('subject_id')

sortedcues = pd.DataFrame(sortedcues)
sortedcues

In [ ]:
# Step 5
# Merging table together
#Changing subject_id from object to int64
change_type ={"subject_id": "int64"}
sortedcues = sortedcues.astype(change_type)
data_types = sortedcues.dtypes
data_types

In [ ]:
sortedcues_gold_std = pd.merge(gold_std, sortedcues, on="subject_id")
sortedcues_gold_std

#sortedcues_gold_std[sortedcues_gold_std['subject_id'] == 43857076]

In [19]:
pd.set_option('display.max_rows', 32)

In [ ]:
# Step 6 
# Getting the total number of people that accessed any image
# Trust or not trust for each task id i.e. image classification on zooniverse
sql = """select c.subject_ids as subject_id, a.annotations->>'value' as zoo_trust 
       from zooniverse_phish_classifications as c, 
    jsonb_array_elements(c.annotations) as a(annotations)
    where a.annotations->>'task' = 'T3'"""

classificationsession = pd.read_sql_query(sql, connection)
classificationsession['subject_id'] = classificationsession['subject_id'].astype('int64')

#classificationsession['zoo_trust_b'] = (classificationsession['zoo_trust'] == "Something's Phishy")

classificationsession[classificationsession['zoo_trust'].isna()]

In [ ]:
# Getting the total number of people who labeled an image
z_total_people_df = classificationsession.groupby('subject_id').apply(lambda x: pd.Series(
               dict(z_no_trust = (x.zoo_trust == "Something's Phishy").sum(),
                    z_trust = (x.zoo_trust == 'Nothing Phishy Here').sum(),
                    z_no_answer = (x.zoo_trust.isna()).sum(),
                   z_total_people = (x.zoo_trust == "Something's Phishy").sum()+ (x.zoo_trust == 'Nothing Phishy Here').sum()))).reset_index('subject_id')
z__total_people_df = pd.DataFrame(z_total_people_df)



#z_total_people_df[z__total_people_df['subject_id'] == 43857076]

z__total_people_df
#gold_std_1= pd.merge(gold_std, z_total_people_df, on= "subject_id")
#gold_std_1

#gold_std_1.to_csv("Z.csv", index=False)


In [ ]:
#z_total_people_df
phase1= pd.read_sql_query("select subject_id_ph1 as subject_id, filename, malicious, gold_std from cybertrust_zooniverse_datamatch", connection)

phase_2= pd.merge(phase1, z_total_people_df, on= "subject_id")
phase_2

#phase_2.to_csv("All_Images_with_malicious_or_not.csv", index=False)


In [ ]:
# Merging the Databases together
sortedcues_total_people = pd.merge(sortedcues_gold_std, z_total_people_df, on= "subject_id")
sortedcues_total_people



In [61]:
#sortedcues_total_people.to_csv("Distinct_people_cue_gld_std.csv", index=False)

Table 1

In [ ]:
# Step 8
# Sorting data into percentage
# The sum method is used at the end of all the calculations to remove formating problems
z_cue_percentages_by_people = sortedcues_total_people.groupby("subject_id").apply(lambda x: pd.Series(
    dict( filename = x.filename.sum(),
          malicious = (x.malicious.sum()  == 1),
          gold_std = (x.gold_std.sum() == 1),
          Invalid_Domain_Sender_Ratio = format((x.Invalid_Domain_or_Sender / x.z_total_people).sum(),'.2%'),
          Potential_Malicious_Links_Ratio = format((x.Potent_Mal_Links / x.z_total_people).sum(), '.2%'),
          Poor_Spelling_or_Grammar_Ratio = format((x.Spelling_or_Grammar / x.z_total_people).sum(), '.2%'),
          Appeal_to_Greed_Ratio = format((x.Appeal_to_Greed / x.z_total_people).sum(), '.2%'),
          Appeal_to_Urgency_Ratio = format((x.Appeal_to_Urgency / x.z_total_people).sum(), '.2%'),
          Appeal_to_Authority_Ratio = format((x.Appeal_to_Authority / x.z_total_people).sum(), '.2%'),
          Other_Phishy_Findings_Ratio = format((x.Other_Phishy_Findings / x.z_total_people).sum(), '.2%'),                             
          Total_People = x.z_total_people.sum()
    ))).reset_index("subject_id")

z_cue_percentages_by_people = pd.DataFrame(z_cue_percentages_by_people)

z_cue_percentages_by_people

CSV for Table 1

In [ ]:
#z_cue_percentages_by_people.to_csv("Cue_perc_by_people.csv", index=False)

Beginning of Version 1.01

In [ ]:
# Step 1
# Getting the total number of none duplicate cues for every image
z_total_cues = sortedcues.groupby("subject_id").apply(lambda x: pd.Series(
    dict(
        total_cues_for_image = (x.Invalid_Domain_or_Sender + x.Potent_Mal_Links + x.Spelling_or_Grammar + x.Appeal_to_Greed + x.Appeal_to_Urgency + x.Appeal_to_Authority + x.Other_Phishy_Findings).sum()
    ))).reset_index("subject_id")


In [ ]:
# Step 2 
# Mergeing the none duplicate cues given and the total number of cues given for a image
sortedcues_total_cue = pd.merge(sortedcues_gold_std, z_total_cues, on="subject_id")
sortedcues_total_cue

Table 2

In [ ]:
# Step 3 
# Creating the ratio table 
# Cue type / total cues for image
z_cue_percentages_by_cue = sortedcues_total_cue.groupby("subject_id").apply(lambda x: pd.Series(
    dict( filename = x.filename.sum(),
          malicious = (x.malicious.sum()  == 1),
          gold_std = (x.gold_std.sum() == 1),
          Invalid_Domain_Sender_Ratio = format((x.Invalid_Domain_or_Sender / x.total_cues_for_image).sum(),'.2%'),
          Potential_Malicious_Links_Ratio = format((x.Potent_Mal_Links / x.total_cues_for_image).sum(), '.2%'),
          Poor_Spelling_or_Grammar_Ratio = format((x.Spelling_or_Grammar / x.total_cues_for_image).sum(), '.2%'),
          Appeal_to_Greed_Ratio = format((x.Appeal_to_Greed / x.total_cues_for_image).sum(), '.2%'),
          Appeal_to_Urgency_Ratio = format((x.Appeal_to_Urgency / x.total_cues_for_image).sum(), '.2%'),
          Appeal_to_Authority_Ratio = format((x.Appeal_to_Authority / x.total_cues_for_image).sum(), '.2%'),
          Other_Phishy_Findings_Ratio = format((x.Other_Phishy_Findings / x.total_cues_for_image).sum(), '.2%'),                             
          Total_Cues_for_Image = x.total_cues_for_image.sum()
    ))).reset_index("subject_id")

z_cue_percentages_by_cue = pd.DataFrame(z_cue_percentages_by_cue)

z_cue_percentages_by_cue


CSV for Table 2

In [ ]:

#z_cue_percentages_by_cue.to_csv("Cue_perc_by_cue.csv", index=False)

Beginning of Verision 1.02

In [ ]:
# Displaying zooniverseclassification dataframe before sorting it
zooniverseclassification


In [ ]:
#Creating a new dataframe 
userboundingboxpos = pd.DataFrame(columns=["user_labeled_image", "subject_id","x_pos", "y_pos", "width", "height", "cue"])

# Sorting the data store in zooniverse classifications into the userboundingboxpos dataframe
for user_labeled_image in range(0, len(zooniverseclassification["user_answers"])):
    current_string = zooniverseclassification["user_answers"][user_labeled_image]
    json_object = json.loads(current_string)
    
    # Holds the cues that a person put on a single image during their session
    # temp_list holds a tuple
    # the tuple are structured (cue type, x_pos, y_pos, width, height)
    temp_list = []
    cue_is_unique = True

    for item in json_object:
        

        for cue in range(0, len(temp_list)):
                
            # This compares the current cue's type being added with cues' that have already been added to the dataframe from the user
            # Checking to see if the cue's are of the same type
            if item["tool_label"] == temp_list[cue][0]:
                # if the cues are of the same type, check that they are intersecting by Separating axis test
                if (
                    (item['x'] > (temp_list[cue][1] + temp_list[cue][3])) and 
                    (temp_list[cue][1] > (item['x'] + item['width'])) and
                    (item['y'] > (temp_list[cue][2] + temp_list[cue][4])) and
                    (temp_list[cue][2] > (item['y'] + item['height']))
                ):
                    cue_is_unique = False
        
        # adds the cue to the data frame if it is unique        
        if cue_is_unique:
            temp_list.append((item["tool_label"], item["x"], item["y"], item["width"],item["height"]))
            
            data_input = {"user_labeled_image": user_labeled_image, 
                            "subject_id": zooniverseclassification["subject_id"][user_labeled_image], 
                            "x_pos": item["x"],
                            "y_pos": item["y"],
                            "width": item["width"],
                            "height": item["height"],
                            "cue": item["tool_label"]}   
        
            userboundingboxpos.loc[len(userboundingboxpos.index)] = data_input

userboundingboxpos

In [ ]:
# Grouping all the cues together
V_subject_cues = userboundingboxpos.groupby("subject_id").apply(lambda x: pd.Series(
    dict(Invalid_Domain_or_Sender = (x.cue == "Invalid Domain or Sender").sum(),
         Potent_Mal_Links = (x.cue == "Potentially Malicious Link").sum(),
         Spelling_or_Grammar = (x.cue == "Poor Spelling or Grammar").sum(),
         Appeal_to_Greed = (x.cue == "Appeal to Action-Greed").sum(),
         Appeal_to_Urgency = (x.cue == "Appeal to Action-Urgency").sum(),
         Appeal_to_Authority = (x.cue == "Appeal to Action-Authority").sum(),
         Other_Phishy_Findings = (x.cue == "Other Phishy Findings").sum()
        ))).reset_index("subject_id")

V_subject_cues = pd.DataFrame(V_subject_cues)

V_subject_cues

In [ ]:
# Merging with gold standard
change_type ={"subject_id": "int64"}
V_subject_cues = V_subject_cues.astype(change_type)

V_gold_std = pd.merge(gold_std, V_subject_cues, on='subject_id')
V_gold_std

In [ ]:
# Getting total number of cues
V_total_cues = V_gold_std.groupby("subject_id").apply(lambda x: pd.Series(
    dict(
        total_cues_for_image = (x.Invalid_Domain_or_Sender + x.Potent_Mal_Links + x.Spelling_or_Grammar + x.Appeal_to_Greed + x.Appeal_to_Urgency + x.Appeal_to_Authority + x.Other_Phishy_Findings).sum()
    ))).reset_index("subject_id")

V_total_cues

In [ ]:
V_gold_std_total_cues = pd.merge(V_gold_std, V_total_cues, on='subject_id')
V_gold_std_total_cues

Table 3

In [ ]:
# Final Table
V_cue_ratio = V_gold_std_total_cues.groupby("subject_id").apply(lambda x: pd.Series(
    dict( filename = x.filename.sum(),
          malicious = (x.malicious.sum()  == 1),
          gold_std = (x.gold_std.sum() == 1),
          Invalid_Domain_Sender_Ratio = format((x.Invalid_Domain_or_Sender / x.total_cues_for_image).sum(),'.2%'),
          Potential_Malicious_Links_Ratio = format((x.Potent_Mal_Links / x.total_cues_for_image).sum(), '.2%'),
          Poor_Spelling_or_Grammar_Ratio = format((x.Spelling_or_Grammar / x.total_cues_for_image).sum(), '.2%'),
          Appeal_to_Greed_Ratio = format((x.Appeal_to_Greed / x.total_cues_for_image).sum(), '.2%'),
          Appeal_to_Urgency_Ratio = format((x.Appeal_to_Urgency / x.total_cues_for_image).sum(), '.2%'),
          Appeal_to_Authority_Ratio = format((x.Appeal_to_Authority / x.total_cues_for_image).sum(), '.2%'),
          Other_Phishy_Findings_Ratio = format((x.Other_Phishy_Findings / x.total_cues_for_image).sum(), '.2%'),                             
          Total_Cues_for_Image = x.total_cues_for_image.sum()
    ))).reset_index("subject_id")

V_cue_ratio

Getting Dataframe for 


In [ ]:
sortedcues

In [ ]:
z_total_people_df

In [ ]:
total_images_labeled_with_people= pd.merge(sortedcues, z_total_people_df, on='subject_id')
total_images_labeled_with_people

In [ ]:
all_cue_over_people_ratio = total_images_labeled_with_people.groupby("subject_id").apply(lambda x: pd.Series(
    dict(
          Invalid_Domain_Sender_Ratio = format((x.Invalid_Domain_or_Sender / x.z_total_people).sum(),'.2%'),
          Potential_Malicious_Links_Ratio = format((x.Potent_Mal_Links / x.z_total_people).sum(), '.2%'),
          Poor_Spelling_or_Grammar_Ratio = format((x.Spelling_or_Grammar / x.z_total_people).sum(), '.2%'),
          Appeal_to_Greed_Ratio = format((x.Appeal_to_Greed / x.z_total_people).sum(), '.2%'),
          Appeal_to_Urgency_Ratio = format((x.Appeal_to_Urgency / x.z_total_people).sum(), '.2%'),
          Appeal_to_Authority_Ratio = format((x.Appeal_to_Authority / x.z_total_people).sum(), '.2%'),
          Other_Phishy_Findings_Ratio = format((x.Other_Phishy_Findings / x.z_total_people).sum(), '.2%'),                             
          Total_People = x.z_total_people.sum()

    ))).reset_index("subject_id")

all_cue_over_people_ratio


In [ ]:
# Select only the columns with percentages
percentage_columns = all_cue_over_people_ratio.iloc[:, 1:8]

# Get the name of the column with the maximum percentage for each row
column_with_max_percentage = percentage_columns.idxmax(axis=1)

# Add the result as a new column to the dataframe
all_cue_over_people_ratio['Column_With_Max_Percentage'] = column_with_max_percentage

all_cue_over_people_ratio

In [ ]:
column_counts = all_cue_over_people_ratio['Column_With_Max_Percentage'].value_counts()

# Create a new dataframe from the counts
count_df = pd.DataFrame({'Column_Name': column_counts.index, 'Count': column_counts.values})

# Calculate the total count for all rows
total_count = count_df['Count'].sum()

# Add a new column for the ratio in count_df
count_df['Ratio'] = count_df['Count'] / total_count

# Display the new dataframe
count_df

In [ ]:

#count_df.to_csv("Cue_prec_by_people_freq.csv", index=False)


In [ ]:
sortedcues_total_people

In [ ]:
z_trust_no_trust = classificationsession.groupby('subject_id').apply(lambda x: pd.Series(
               dict(z_trust = (x.zoo_trust_b ==  True).sum(),
                    z_no_trust =(x.zoo_trust_b == False).sum()
                    ))).reset_index('subject_id')
z_trust_no_trust = pd.DataFrame(z_trust_no_trust)

z_trust_no_trust[z_trust_no_trust['subject_id'] == 43857918	
]


In [ ]:
sortedcues_total_people = pd.merge(sortedcues_total_people, z_trust_no_trust, on= 'subject_id')
sortedcues_total_people

In [ ]:
sortedcues_total_people.to_csv("Distinct_people_cue_gld_std.csv", index=False)